# Using Geoprocessing tools

This sample notebooks shows how geoprocessing tools can be used in an analysis. The analysis below uses a geoprocessing tool to  deduce the path that the debris of a crashed airplane would take if it went down at different places in the ocean. It should be noted that this tool is for the purpose of demonstrating GIS functionality and the results should not be treated as authorotative. It is using a sample tool is for demonstration of GIS functionality only and excludes wind or atmospheric conditions, which are critical for a real-world analysis.

In [1]:
from arcgis.gis import GIS

In [2]:
gis = GIS()
map = gis.map()
map

There have been speculations that the unfortunate disappearance of MH370 is tied to Diego Garcia, in addition to the search location in southern Indian Ocean west of Australia. The code below plots the locations of these in the map:

In [3]:
diegogarcia = gis.tools.geocoder.find_best_match("Diego Garcia")
map.center = diegogarcia

In [4]:
map.draw(gis.tools.geocoder.find_best_match("La Reunion"), {"title": "Reunion Island", "content": "Debris found"})
map.draw(diegogarcia, {"title": "Diego Garcia", "content": "Naval Support Facility Diego Garcia"})
map.draw([-43.5, 90.5], {"title":"Search Location", "content":"Predicted crash location"})

#Source: http://www.amsa.gov.au/media/incidents/images/DIGO_00718_01_14.jpg
map.add_layer({"type":"FeatureLayer", "url" : "http://services.arcgis.com/WQ9KVmV6xGGMnCiQ/arcgis/rest/services/MH370Search/FeatureServer/1"})

We can search ArcGIS Online for content related to the missing airliner, and bring in layers that we want to use for our analysis and visualization:

In [5]:
from IPython.display import display

mh370items = gis.content.search("MH370", "feature service", max_items=6)
for item in mh370items:
    display(item)

<Item title:"MH370_Flight_Path" type:Feature Service owner:bflanagan_bureau>

<Item title:"Arc" type:Feature Service owner:bflanagan_bureau>

<Item title:"MH370Search" type:Feature Service owner:bflanagan_bureau>

<Item title:"ActualFlightRoute" type:Feature Service owner:bflanagan_bureau>

<Item title:"SatelliteLocation" type:Feature Service owner:bflanagan_bureau>

<Item title:"mh370debris" type:Feature Service owner:bflanagan_bureau>

Let's add the layers that we want to use for our analysis to the map:

In [6]:
map.add_layer(mh370items[0])
map.add_layer(mh370items[4])
map.add_layer(mh370items[5])

In [7]:
map.zoom = 6

# Using geoprocessing tool

We search ArcGIS Online for the Ocean Currents geoprocessing tool that calculates the path of an unpowered vessel drifiting in ocean currents from a user defined point. This tool is for demonstration of GIS functionality only and excludes wind or atmospheric conditions, which are critical for a real-world analysis:

In [8]:
gpsvcs = gis.content.search("Ocean Currents", "geoprocessing service", max_items=1)
for gpsvc in gpsvcs:
    display(gpsvc)

<Item title:"Ocean Currents" type:Geoprocessing Service owner:esri_apac>

In [9]:
from arcgis.tools import GeoprocessingTool

gptool = GeoprocessingTool(gpsvcs[0])

URL: http://sampleserver1.arcgisonline.com/ArcGIS/rest/Services/Specialty/ESRI_Currents_World/GPServer
Task: MessageInABottle
Input_Point : GPFeatureRecordSetLayer
Days : GPDouble


When we click the map, we want to invoke the geoprocessing tool and pass in the clicked location. We can do this by adding an event listener to the map using map.on_click() that calls the tool with the clicked location. The ArcGIS Python API dynamically adds methods (in this case, the message_in_a_bottle method) based on the tool that is contructed. The method is added with documentation strings that can be queried in the notebook using intellisense.

This tool would return the paths that debris would take if the plane went down at that path, which we draw on the map using map.draw() in the code below:

In [10]:
def do_analysis(m, g):
    ret = gptool.message_in_a_bottle(g, 150)
    for geom in ret:
        map.draw(geom)

map.on_click(do_analysis)

Click around on the map, near Diego Garcia as well as the search location to find the paths the debris could have taken and if could it have reached La Reunion if the plane indeed went down in these locations.